In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import edunn as nn
from edunn import utils

# Capa Convolucional 2D

Siguiendo la misma idea, podemos extender el concepto de convolución sobre matrices. Es decir, una convolución en 2 dimensiones. Esto nos sirve para imágenes en escala de grises, RGB y por lotes.

## Método Forward

Una capa convolucional 2D se forma aplicando una operación de convolución 2D a la entrada, seguida de una adición de Bias si se desea. La convolución es una operación sobre dos funciones $x$ y $k$, que produce una tercera función que puede ser interpretada como una versión de $x$ "filtrada" por $w$. Si bien la convolución se define en forma continua, a nosotros nos interesa la versión discreta.

Entonces, dado un batch de imágenes de entrada de shape $(N,C_{\text{in}},H,W)$ y un batch de filtros de shape $(M,C_{\text{out}},F,F)$, la operación de convolución se puede expresar como:

$$
y[l,m,i,j] = (x \circledast w) = \sum_{a}\sum_{b} w[m,:,a,b] \cdot x[l,:,i+a,j+b]
$$

Donde $N$ es la cantidad de imágenes del lote, $C_{\text{in/out}}$ es el número de canales de la imagen de entrada y feature map de salida (respectivamente), $M$ es la cantidad de feature maps deseados y $F$ es el tamaño del kernel.

En esta fórmula se ve que las sumas se realizan sobre las dimensiones del kernel y sobre los $C$ canales a la vez, además se asume que el **stride** (paso) es de 1 y el **padding** (relleno) es de 0. El término de bias se suma después de la operación de convolución.

* Las funciones de activación pueden funcionar de manera matricial, aplicandose a cada valor de $y[k,i,j]$ sin necesidad de cambio alguno.

|||
|:-:|:-:|
|![conv2d_example.gif](img/conv2d_example.gif)|![featue_maps.gif](img/featue_maps.gif)|

## Hiperparámetros

El tamaño de la matriz resultante $y$ y está estrictamente relacionado por los siguientes parámetros:

* `kernel_size`: es el tamaño del filtro utilizado.
* `stride`: es el número de saltos que da el filtro cada vez que se aplica. 
* `padding`: es la cantidad de píxeles rellenos con cero en los bordes.
   * Aplicar el filtro de forma discreta ocasiona dos problemas:
       * Pérdida de información en los bordes.
       * Reducción del tamaño final del vector.
   <!-- * Se utiliza para obtener una imagen resultante de un tamaño buscado. -->

|strides|padding|
|:-:|:-:|
|![stride.gif](img/stride.gif)|![padding.gif](img/padding.gif)|

## Shapes

El tamaño de la salida $y$ (es decir, los feature maps resultantes) depende del tamaño $H_{in} \times W_{in}$ de la imagen de entrada, el tamaño del kernel $F$, el stride (paso) $S$ y el padding (relleno) $P$. La fórmula general para calcular la altura $H_{out}$ y el ancho $W_{out}$ de la salida en una capa convolucional es:

$$
\begin{aligned}
A_{out} &= \left\lfloor \frac{A_{in} - F + 2P}{S} \right\rfloor + 1 & \text{donde $A \in \{W, H\}$} \\
\end{aligned}
$$

Por lo tanto, el shape de la salida $y$ es $(N, M, H_{out}, W_{out})$. El shape del bias depende únicamente del número de feature maps $M$. Notar que:

- El tamaño del batch en la salida es igual al tamaño del batch en la entrada. Esto se debe a que cada imagen en el batch se procesa de forma independiente.
- El tamaño de los canales en la salida es igual al tamaño del batch en el kernel. Esto se debe a que cada canal de salida se genera al convolucionar la entrada con un kernel diferente.

## Implementación

Similarmente como sucede con el Modelo Lineal, en lugar de ver a $y$ como $x \circledast w + b$, podemos verlo como `x -> Convolution -> Bias -> y`. Es decir, como una sucesión de capas, donde cada una transforma la entrada `x` hasta obtener la salida `y`.

En términos de código, el método `forward` de un modelo de `Convolution` es la composición de las funciones `forward` de las capas `Convolution` y `Bias`:

```python
y_conv = conv2d(x)  # Implementar la función dentro del mismo modelo
y = bias.forward(y_conv)
```

> TIP: la operación de convolución también se puede representar como la **cross-correlation** entre $x$ y $\text{rot}_{180^\circ } \left \{ w \right \}$, es decir:
> $$
y = \left( \begin{bmatrix} x_{11} & x_{12} & x_{13} \\ x_{21} & x_{22} & x_{23} \\ x_{31} & x_{32} & x_{33} \end{bmatrix}  \circledast \begin{bmatrix} w_{11} & w_{12} \\ w_{21} & w_{22} \end{bmatrix} \right)
\Leftrightarrow
\left( \begin{bmatrix} x_{11} & x_{12} & x_{13} \\ x_{21} & x_{22} & x_{23} \\ x_{31} & x_{32} & x_{33} \end{bmatrix} \star \begin{bmatrix} w_{22} & w_{21} \\ w_{12} & w_{11} \end{bmatrix} \right) = y
$$

Implementa el método `forward` del modelo `Convolution` en el archivo `edunn/models/convolution.py`.

In [3]:
np.random.seed(123)

x = np.random.rand(2,3,7,7)
w = np.random.rand(4,3,5,5)
                       
stride,padding=1,0
kernel_initializer = nn.initializers.Constant(w)

layer=nn.Convolution2D(x.shape[1],w.shape[0],kernel_size=w.shape[2:],stride=stride,padding=padding,kernel_initializer=kernel_initializer)

In [4]:
y = layer.forward(x)
y, y.shape

(array([[[[18.91400336, 17.74220249, 17.04932403],
          [18.39506163, 18.76882808, 17.30981571],
          [19.12504344, 18.17784254, 17.33288676]],
 
         [[18.24159502, 18.27507403, 17.56270967],
          [20.4349386 , 18.93949341, 16.16328647],
          [17.90154639, 18.3689272 , 18.46915102]],
 
         [[19.28960258, 19.61526087, 17.85885938],
          [19.63929637, 19.83030544, 17.68385589],
          [19.17803285, 19.59178382, 18.38052667]],
 
         [[19.22944905, 19.07249265, 18.33651964],
          [21.05243436, 20.29708406, 17.76695975],
          [18.62752827, 19.51223509, 18.87998511]]],
 
 
        [[[17.0583646 , 17.35510848, 16.85042324],
          [16.82504731, 18.6930386 , 18.29630443],
          [17.30483294, 15.6852319 , 16.89828771]],
 
         [[19.46707539, 19.90947773, 18.34971515],
          [17.85584063, 17.68460619, 18.48858847],
          [18.25931409, 17.00230864, 16.81465981]],
 
         [[20.86567419, 18.45006639, 17.71231939],
          

## Método Backward

La explicación de su cálculo es sencilla a partir de un ejemplo, supongamos que $x \in \mathbb{R}^{(3\times 3)}$ y $w \in \mathbb{R}^{(2\times 2)}$, de modo los $y_{ij}$ se definen como:

$$\begin{aligned}
y_{11} &= x_{11} \cdot w_{11} + x_{12} \cdot w_{12} + x_{21} \cdot w_{21} + x_{22} \cdot w_{22} \\
y_{12} &= x_{12} \cdot w_{11} + x_{13} \cdot w_{12} + x_{22} \cdot w_{21} + x_{23} \cdot w_{22} \\
y_{21} &= x_{21} \cdot w_{11} + x_{22} \cdot w_{12} + x_{31} \cdot w_{21} + x_{32} \cdot w_{22} \\
y_{22} &= x_{22} \cdot w_{11} + x_{23} \cdot w_{12} + x_{32} \cdot w_{21} + x_{33} \cdot w_{22}
\end{aligned}$$

### `δE/δw`

El cálculo de los gradientes del error $E$ con respecto al filtro $w$ (vector de pesos aprendido) se puede hacer derivando parcialmente como se explicó en guías anteriores:

$$
\begin{aligned}
\frac{\partial E}{\partial w_{11}} &= \frac{\partial E}{\partial y_{11}} \frac{\partial y_{11}}{\partial w_{11}} + \frac{\partial E}{\partial y_{12}} \frac{\partial y_{12}}{\partial w_{11}} + \frac{\partial E}{\partial y_{21}} \frac{\partial y_{21}}{\partial w_{11}} + \frac{\partial E}{\partial y_{22}} \frac{\partial y_{22}}{\partial w_{11}}
\\
\frac{\partial E}{\partial w_{12}} &= \frac{\partial E}{\partial y_{11}} \frac{\partial y_{11}}{\partial w_{12}} + \frac{\partial E}{\partial y_{12}} \frac{\partial y_{12}}{\partial w_{12}} + \frac{\partial E}{\partial y_{21}} \frac{\partial y_{21}}{\partial w_{12}} + \frac{\partial E}{\partial y_{22}} \frac{\partial y_{22}}{\partial w_{12}}
\\
\frac{\partial E}{\partial w_{21}} &= \frac{\partial E}{\partial y_{11}} \frac{\partial y_{11}}{\partial w_{21}} + \frac{\partial E}{\partial y_{12}} \frac{\partial y_{12}}{\partial w_{21}} + \frac{\partial E}{\partial y_{21}} \frac{\partial y_{21}}{\partial w_{21}} + \frac{\partial E}{\partial y_{22}} \frac{\partial y_{22}}{\partial w_{21}}
\\
\frac{\partial E}{\partial w_{22}} &= \frac{\partial E}{\partial y_{11}} \frac{\partial y_{11}}{\partial w_{22}} + \frac{\partial E}{\partial y_{12}} \frac{\partial y_{12}}{\partial w_{22}} + \frac{\partial E}{\partial y_{21}} \frac{\partial y_{21}}{\partial w_{22}} + \frac{\partial E}{\partial y_{22}} \frac{\partial y_{22}}{\partial w_{22}}
\end{aligned}
$$

Donde utilizando las ecuaciones anteriormente definidas al principio de esta celda, es fácil ver que:

$$
\begin{aligned}
\frac{\partial E}{\partial w_{11}} &= \frac{\partial E}{\partial y_{11}} x_{11} + \frac{\partial E}{\partial y_{12}} x_{12} + \frac{\partial E}{\partial y_{21}} x_{21} + \frac{\partial E}{\partial y_{22}} x_{22}
\\
\frac{\partial E}{\partial w_{12}} &= \frac{\partial E}{\partial y_{11}} x_{12} + \frac{\partial E}{\partial y_{12}} x_{13} + \frac{\partial E}{\partial y_{21}} x_{22} + \frac{\partial E}{\partial y_{22}} x_{23}
\\
\frac{\partial E}{\partial w_{21}} &= \frac{\partial E}{\partial y_{11}} x_{21} + \frac{\partial E}{\partial y_{12}} x_{22} + \frac{\partial E}{\partial y_{21}} x_{31} + \frac{\partial E}{\partial y_{22}} x_{32}
\\
\frac{\partial E}{\partial w_{22}} &= \frac{\partial E}{\partial y_{11}} x_{22} + \frac{\partial E}{\partial y_{12}} x_{23} + \frac{\partial E}{\partial y_{21}} x_{32} + \frac{\partial E}{\partial y_{22}} x_{33}
\end{aligned}
$$

Tiene un patrón idéntico al de una convolución entre $x$ y $\frac{\delta E}{\delta y}$:

$$
\frac{\delta E}{\delta w} = \begin{bmatrix} x_{11} & x_{12} & x_{13} \\ x_{21} & x_{22} & x_{23} \\ x_{31} & x_{32} & x_{33} \end{bmatrix}  \circledast \begin{bmatrix} \frac{\partial E}{\partial y_{11}} & \frac{\partial E}{\partial y_{12}} \\ \frac{\partial E}{\partial y_{21}} & \frac{\partial E}{\partial y_{22}} \end{bmatrix} = x \circledast \frac{\delta E}{\delta y}
$$

### `δE/δx`

El razonamiento para el cálculo de los gradientes del $E$ con respecto a la imagen de entrada $x$ es el mismo que antes:

$$
\begin{aligned}
\frac{\partial E}{\partial x_{11}}&=\frac{\partial E}{\partial y_{11}}w_{11}+\frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{22}}\;\:\, 0 \; 
\\
\frac{\partial E}{\partial x_{12}}&=\frac{\partial E}{\partial y_{11}}w_{12}+\frac{\partial E}{\partial y_{12}}w_{11}+\frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{22}}\;\:\, 0 \; 
\\
\frac{\partial E}{\partial x_{13}}&=\frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{12}}w_{12}+\frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{22}}\;\:\, 0 \; 
\\
\frac{\partial E}{\partial x_{21}}&=\frac{\partial E}{\partial y_{11}}w_{21}+\frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{21}}w_{11}+\frac{\partial E}{\partial y_{22}}\;\:\, 0 \; 
\\
\frac{\partial E}{\partial x_{22}}&=\frac{\partial E}{\partial y_{11}}w_{22}+\frac{\partial E}{\partial y_{12}}w_{21}+\frac{\partial E}{\partial y_{21}}w_{12}+\frac{\partial E}{\partial y_{22}}w_{11} 
\\
\frac{\partial E}{\partial x_{23}}&=\frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{12}}w_{22}+\frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{22}}w_{11} 
\\
\frac{\partial E}{\partial x_{31}}&=\frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{21}}w_{21}+\frac{\partial E}{\partial y_{22}}\;\:\, 0 \; 
\\
\frac{\partial E}{\partial x_{32}}&=\frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{21}}w_{22}+\frac{\partial E}{\partial y_{22}}w_{21} 
\\
\frac{\partial E}{\partial x_{33}}&=\frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{22}}w_{22} 
\end{aligned}
$$

Tal cálculo se puede automatizar mediante el operador de **same-convolution**, en la que se aplica padding a la entrada de tal manera que la dimensión de la salida sea la misma que la de la entrada. Además, para poder representarlo de tal manera, es obligatorio que la matriz de filtros $w$ se rote 180°. 

$$
\frac{\delta E}{\delta x} = \begin{bmatrix} \frac{\partial E}{\partial y_{11}} & \frac{\partial E}{\partial y_{12}} \\ \frac{\partial E}{\partial y_{21}} & \frac{\partial E}{\partial y_{22}} \end{bmatrix} \circledast \begin{bmatrix} w_{22} & w_{21} \\ w_{12} & w_{11} \end{bmatrix} = \left. \left[ \frac{\delta E}{\delta y} \circledast \text{rot}_{180^\circ } \left \{ w \right \} \right] \right|_{\text{same-padding}}
$$

![conv2d_backward.gif](img/conv2d_backward.gif)

La formula para lograr el padding adecuado queda como ejercicio para el lector deducirla (puede encontrar ayuda en el blog [cs231n](https://cs231n.github.io/convolutional-networks/)).

> NOTA: Tener en cuenta que en cálculo del backward el stride se establece a 1, independientemente del stride que se utilizó en el forward. Esto se debe a que en la propagación hacia atrás se calcula el gradiente de la función de error con respecto a cada elemento de la entrada y el kernel, por lo que es necesario considerar cada posición posible del kernel sobre la entrada.

<!-- La diferencia entre una 'full convolution' y una 'same convolution' radica en cómo se maneja el 'padding' (relleno) alrededor de la entrada:

- **Full Convolution**: En una convolución completa, no se aplica 'padding' a la entrada. Como resultado, la dimensión de la salida es mayor que la de la entrada. Específicamente, si la entrada es de tamaño $n \times n$ y el filtro es de tamaño $f \times f$, entonces la salida será de tamaño $(n+f-1) \times (n+f-1)$.

- **Same Convolution**: En una convolución 'same', se aplica 'padding' a la entrada de tal manera que la dimensión de la salida sea la misma que la de la entrada. Es decir, si la entrada es de tamaño $n \times n$, independientemente del tamaño del filtro, la salida también será de tamaño $n \times n$. 

https://stats.stackexchange.com/questions/297678/how-to-calculate-optimal-zero-padding-for-convolutional-neural-networks
https://ai.stackexchange.com/questions/27779/what-is-the-difference-between-same-convolution-and-full-convolution-in-terms-of
-->

In [5]:
# Define el gradiente de la salida
g = np.ones_like(y)

# Propaga el gradiente hacia atrás a través de la convolución
layer_grad = layer.backward(g)
layer_grad

(array([[[[ 1.34111656,  4.72856081,  7.41120998,  8.41821486,
            6.52373009,  3.84108092,  1.49295947],
          [ 3.045064  ,  7.06538423, 12.27498094, 12.74519571,
           12.52338572,  7.31378901,  3.79851025],
          [ 4.97413319, 10.48124245, 18.00389502, 18.38297049,
           18.78367768, 11.26102512,  5.90781646],
          [ 6.13257498,  9.87439054, 16.35681692, 15.44306343,
           18.79576364, 12.31333726,  7.09451576],
          [ 6.21102484, 10.82021585, 17.00942959, 17.41496378,
           19.52432632, 13.33511259,  6.71855356],
          [ 4.28195565,  7.40435763, 11.28051551, 11.777189  ,
           13.26403436,  9.38787648,  4.60924734],
          [ 1.78239729,  3.28264874,  5.51638364,  6.2988812 ,
            6.72821832,  4.49448342,  1.92958857]],
 
         [[ 1.74328732,  3.14375925,  5.16800132,  5.16121919,
            5.00105137,  2.9768093 ,  1.24030411],
          [ 4.50196146,  7.37367317, 10.87176453, 10.44542892,
           10.67213988

# Comprobaciones con PyTorch

In [6]:
import torch
import torch.nn as tnn

w = torch.from_numpy(w).to(torch.double)
x = torch.from_numpy(x).to(torch.double)

conv = tnn.Conv2d(in_channels=x.shape[1], out_channels=w.shape[0], kernel_size=w.shape[-1], stride=stride, padding=padding, bias=False)
conv.weight.data = w

x.requires_grad = True
conv.weight.requires_grad = True

y_torch = conv(x)
y_torch

tensor([[[[18.9140, 17.7422, 17.0493],
          [18.3951, 18.7688, 17.3098],
          [19.1250, 18.1778, 17.3329]],

         [[18.2416, 18.2751, 17.5627],
          [20.4349, 18.9395, 16.1633],
          [17.9015, 18.3689, 18.4692]],

         [[19.2896, 19.6153, 17.8589],
          [19.6393, 19.8303, 17.6839],
          [19.1780, 19.5918, 18.3805]],

         [[19.2294, 19.0725, 18.3365],
          [21.0524, 20.2971, 17.7670],
          [18.6275, 19.5122, 18.8800]]],


        [[[17.0584, 17.3551, 16.8504],
          [16.8250, 18.6930, 18.2963],
          [17.3048, 15.6852, 16.8983]],

         [[19.4671, 19.9095, 18.3497],
          [17.8558, 17.6846, 18.4886],
          [18.2593, 17.0023, 16.8147]],

         [[20.8657, 18.4501, 17.7123],
          [18.1397, 18.7252, 18.3487],
          [17.5394, 17.3103, 19.4259]],

         [[19.1796, 18.7535, 17.1503],
          [17.8921, 16.7085, 17.9420],
          [17.5999, 16.8651, 17.5964]]]], dtype=torch.float64,
       grad_fn=<Convolut

In [7]:
utils.check_same(y_torch.detach().numpy(),y)

SUCCESS :) Arrays are equal (tolerance 1e-12)


In [8]:
# Define el gradiente de la salida
g = torch.ones_like(y_torch)

# Propaga el gradiente hacia atrás a través de la convolución
y_torch.backward(g)

# Imprime el gradiente de la imagen de entrada y el kernel
print("Gradiente de la entrada (δE/δx):")
print(x.grad, x.grad.shape)
print("\nGradiente del kernel (δE/δw):")
print(conv.weight.grad, conv.weight.grad.shape)

Gradiente de la entrada (δE/δx):
tensor([[[[ 1.3411,  4.7286,  7.4112,  8.4182,  6.5237,  3.8411,  1.4930],
          [ 3.0451,  7.0654, 12.2750, 12.7452, 12.5234,  7.3138,  3.7985],
          [ 4.9741, 10.4812, 18.0039, 18.3830, 18.7837, 11.2610,  5.9078],
          [ 6.1326,  9.8744, 16.3568, 15.4431, 18.7958, 12.3133,  7.0945],
          [ 6.2110, 10.8202, 17.0094, 17.4150, 19.5243, 13.3351,  6.7186],
          [ 4.2820,  7.4044, 11.2805, 11.7772, 13.2640,  9.3879,  4.6092],
          [ 1.7824,  3.2826,  5.5164,  6.2989,  6.7282,  4.4945,  1.9296]],

         [[ 1.7433,  3.1438,  5.1680,  5.1612,  5.0011,  2.9768,  1.2403],
          [ 4.5020,  7.3737, 10.8718, 10.4454, 10.6721,  7.1740,  3.0984],
          [ 6.6720, 11.1750, 17.1003, 16.8620, 17.7775, 11.8522,  5.4184],
          [ 5.6011, 10.0356, 15.1478, 16.9930, 19.0610, 13.9488,  6.5025],
          [ 4.4604, 10.1062, 15.8820, 18.0887, 19.1016, 13.3258,  6.6586],
          [ 2.2904,  6.3049,  9.6534, 11.6721, 11.9962,  8.6477, 

In [9]:
utils.check_same(x.grad.numpy(),layer_grad[0])
utils.check_same(conv.weight.grad.numpy(),layer_grad[1]['w'])

SUCCESS :) Arrays are equal (tolerance 1e-12)
SUCCESS :) Arrays are equal (tolerance 1e-12)


In [10]:
samples = 100
batch_size=2
din=3 # dimensión de entrada
dout=5 # dimensión de salida
input_shape=(batch_size,din,5,5)

# Verificar las derivadas de un modelo de Convolución
# con valores aleatorios de `w`, `b`, y `x`, la entrada
layer=nn.Convolution2D(input_shape[1],dout,kernel_size=(3,3))


utils.check_gradient.common_layer(layer,input_shape,samples=samples,tolerance=1e-5)    

Convolution2D_1 layer:
SUCCESS 28500 partial derivatives checked (100 random input samples)
